# Comparison Example : scores

In this jupyter notebook we present the implementation of the gridness score as defined by by Sargolini et al 2006 https://pubmed.ncbi.nlm.nih.gov/16675704/. 

We conduct an autocorrelation analysis of the smoothed spatial rate map. From this analysis, we identified six peaks closest to the central peak and encompassed them within a ring, excluding the central peak. The purpose of this step was to examine the periodicity between these fields . Subsequently, we performed a rotational autocorrelation of this ring and observed the correlations' periodic patterns. To quantify the gridness, we computed a gridness score. This score was determined as the difference between the lowest correlation observed at 60 or 120 degrees of rotation and the highest correlation observed at 30, 90, or 150 degrees of rotation. The results of this analysis are presented as 'gridness measure 1

Generraly a score larger than [0.2] clasify as a grid cell.


 
The code was appadted from:

https://github.com/rhayman/ephysiopy/blob/12ee57a858161daa44546b75853a3333097ceb8a/ephysiopy/common/fieldcalcs.py#L743

We note that other measures have been developped to compute the gridness of a neural representation. We aim to implement a larger variety of score over time.



In [29]:
import sys
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from neuralplayground.arenas import Wernle2018, MergingRoom, Sargolini2006, Simple2D, BasicSargolini2006,Hafting2008
from neuralplayground.agents import Weber2018, Stachenfeld2018, RandomAgent, LevyFlightAgent
from neuralplayground.experiments import Wernle2018Data, Hafting2008Data, Sargolini2006Data
from neuralplayground.comparison import GridScorer


In [30]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Experimental results 

We compute the gridness score for various experimental recordings 

###  Sargolini 

The get_scores function gives as out-put the spacial auto correlogram as well as various measures extracted from a spatial auto correlogram, including the grid score. 



In [3]:
sargolini_data = Sargolini2006Data(verbose=False)
env = Sargolini2006(time_step_size=1,
                         agent_step_size=None)

# Change the recording_index=1 to see differernt celss
r_out_im,x_bin, y_bin = env.recording_tetr(recording_index=2)
print(x_bin)
GridScorer_Sargolini2006 = GridScorer(x_bin)
GridScorer_Sargolini2006.plot_grid_score(r_out_im=r_out_im, plot= True)
score = GridScorer_Sargolini2006 .get_scores(r_out_im)

print('GridScorer_Sargo')
print(score)


###  Hafting


In [4]:
hafting_data = Hafting2008Data(verbose=False)
env = Hafting2008(time_step_size=1,
                         agent_step_size=None)
r_out_im,x_bin, y_bin = env.recording_tetr(recording_index=6)
print(y_bin)
GridScorer_Hafting2008 = GridScorer(x_bin )
GridScorer_Hafting2008.plot_grid_score(r_out_im=r_out_im, plot= True)
score = GridScorer_Hafting2008.get_scores(r_out_im)
print(score)

The plot_sac function returns a plot of the auto correlogram.

### Wernle

In [5]:
wernle_data =  Wernle2018Data()
h, x_bin, y_bin = wernle_data.recording_tetr()
r_out_im = h.reshape((x_bin.size, y_bin.size))
GridScorer_Wernle = GridScorer(x_bin.size-1)
score_Wernle = GridScorer_Wernle.get_scores(r_out_im)
GridScorer_Wernle.plot_sac(score_Wernle[0])
print('GridScorer_SR')
print(score)


In [6]:
env = Wernle2018(time_step_size=1,
                         agent_step_size=None)
ax = env.plot_recording_tetr(recording_index=40)

In [7]:
print('GridScorer_SR')
print(score_Wernle[1])

## Agents results 

We compute the gridness score for various Agents output recordings 

### Weber

In [27]:
# Parameters for a square environment
room_width = [-10,10]
room_depth = [-10,10]
env_name = "env_example"
time_step_size = 1
agent_step_size = 0.5

# Init environment
envsimple = Simple2D(arena_x_limits = room_width,
                     arena_y_limits = room_depth,
                     time_step_size = time_step_size,
                     agent_step_size = agent_step_size)

exc_eta = 2e-4
inh_eta = 8e-4
model_name = "model_example"
sigma_exc = np.array([0.05, 0.05])
sigma_inh = np.array([0.1, 0.1])
Ne = 4900
Ni = 1225
Nef = 1
Nif = 1
alpha_i = 1
alpha_e = 1
we_init = 1.0
wi_init = 1.5
agent_step_size = 0.1
roh = 1
agent_Webber = Weber2018(model_name=model_name, exc_eta=exc_eta, inh_eta=inh_eta, sigma_exc=sigma_exc,
                  sigma_inh=sigma_inh, Ne=Ne, Ni=Ni, agent_step_size=agent_step_size, ro=roh,
                  Nef=Nef, Nif=Nif, room_width=envsimple.room_width, room_depth=envsimple.room_depth,
                  alpha_i=alpha_i, alpha_e=alpha_e, we_init=we_init, wi_init=wi_init)

plot_every = 1000
total_iters = 0
n_steps = 4000
obs, state = envsimple.reset()
for i in tqdm(range(n_steps)):
    # Observe to choose an action, the first to numbers in the observation are the xy position of the agent
    obs = obs[:2]
    action = agent_Webber.act(obs)
    # rate = agent.update()
    agent_Webber.update()
    # Run environment for given action
    obs, state, reward = envsimple.step(action, normalize_step=True)
    total_iters += 1
    if i % plot_every == 0:
        agent_Webber.plot_rate_map()


In [31]:
print('GridScorer_Webber')

r_out_im = agent_Webber.get_rate_map_matrix()
GridScorer_Weber2018 = GridScorer(agent_Webber.resolution_width)
GridScorer_Weber2018.plot_grid_score(r_out_im=r_out_im, plot= True)
score = GridScorer_Weber2018.get_scores(r_out_im)
GridScorer_Weber2018.plot_sac(score[0])
print(score)

### Stachenfeld

In [15]:
agent_step_size = 1
discount = .9
threshold = 1e-6
lr_td = 1e-2
t_episode = 10000
n_episode = 10000
state_density = (1 / agent_step_size)
twoDvalue = True
agent_SR = Stachenfeld2018(discount=discount, t_episode=t_episode, n_episode=n_episode, threshold=threshold, lr_td=lr_td,
               room_width=envsimple.room_width, room_depth=envsimple.room_depth, state_density=state_density, twoD=twoDvalue)
print('GridScorer_SR')
r_out_im=agent_SR.get_rate_map_matrix(eigen_vector=30)

GridScorer_Stachenfeld2018 = GridScorer(agent_SR.resolution_width)
GridScorer_Stachenfeld2018.plot_grid_score(r_out_im=r_out_im, plot= True)
score = GridScorer_Stachenfeld2018.get_scores(r_out_im)

print(score)

### Whittington
Please refer to [TEM_README.md](examples/agent_examples/TEM_README.md) to run the following part

In [20]:
import importlib
import os
import pickle
import pandas as pd
import torch
import matplotlib.pyplot as plt
import neuralplayground.agents.whittington_2020_extras.whittington_2020_analyse as analyse
from neuralplayground.plotting import PlotSim

In [21]:
simulation_id = "TEM_custom_plot_sim"
from neuralplayground.saved_models import fetch_model_path
save_path = fetch_model_path("whittington_2020_in_discritized_objects")
plotting_loop_params = {"n_episode": 5000}

In [22]:
training_dict = pd.read_pickle(os.path.join(os.getcwd(), save_path, "params.dict"))
model_weights = pd.read_pickle(os.path.join(save_path, "agent"))
model_spec = importlib.util.spec_from_file_location("model", save_path + "whittington_2020_model.py")
model = importlib.util.module_from_spec(model_spec)
model_spec.loader.exec_module(model)
params = pd.read_pickle(os.path.join(save_path, "agent_hyper"))
tem = model.Model(params)
tem.load_state_dict(model_weights)
tem.eval()

In [23]:
sim = PlotSim(
    simulation_id=simulation_id,
    agent_class=training_dict["agent_class"],
    agent_params=training_dict["agent_params"],
    env_class=training_dict["env_class"],
    env_params=training_dict["env_params"],
    plotting_loop_params=plotting_loop_params,
)

In [24]:
trained_agent, trained_env = sim.plot_sim(save_path, n_walks=1000, random_state=False, custom_state=[0.5,0.5])

In [25]:
with open(os.path.join(save_path, "NPG_environments.pkl"), "rb") as f:
    environments = pickle.load(f)
with open(os.path.join(save_path, "NPG_model_input.pkl"), "rb") as f:
    model_input = pickle.load(f)

training_dict["params"] = training_dict["agent_params"]
del training_dict["agent_params"]
agent = training_dict["agent_class"](**training_dict["params"])
agent.plot_run(tem, model_input, environments)
agent.plot_rate_map(rate_map_type='g');
agent.plot_rate_map(rate_map_type='p');
rate_map_mat = agent.get_rate_map_matrix(agent.g_rates,1,2)
GridScorer = GridScorer(rate_map_mat.shape[0])
GridScorer.plot_grid_score(r_out_im = rate_map_mat, plot= True)
score = GridScorer.get_scores(rate_map_mat)